# Transformer麻雀AI 開発ワークフロー (緊急調査モード)

**【目的】**
データ生成過程におけるサンプル棄却率が異常に高い問題の原因を特定する。
パーサーの手牌シミュレーションにズレが生じていないかを徹底的に検証する。

## 1. 環境設定

In [ ]:
!pip install mahjong
from google.colab import drive
drive.mount('/content/drive')

# ご指定のプロジェクトパスに移動します
%cd "/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI"

## 2. データ準備
調査対象の牌譜ログをローカル環境にコピーします。

In [ ]:
import os
import zipfile

# ご自身の環境に合わせて、牌譜ログが入ったzipファイル名を指定してください
ZIP_FILE_PATH = 'data.zip'
EXTRACT_DIR = '/content/data'

print(f"Copying '{ZIP_FILE_PATH}' to local environment...")
!cp -f "{ZIP_FILE_PATH}" /content/

print(f"Unzipping files to '{EXTRACT_DIR}'...")
if os.path.isdir(EXTRACT_DIR):
    !rm -rf {EXTRACT_DIR}
os.makedirs(EXTRACT_DIR, exist_ok=True)
with zipfile.ZipFile(f'/content/{ZIP_FILE_PATH}', 'r') as zip_ref:
    zip_ref.extractall(EXTRACT_DIR)

# zipファイル内に'data'ディレクトリがネストされている場合に対応
NESTED_DATA_PATH = os.path.join(EXTRACT_DIR, 'data')
if os.path.isdir(NESTED_DATA_PATH):
    print(f"Correcting nested data directory...")
    !rsync -a {NESTED_DATA_PATH}/ {EXTRACT_DIR}/
    !rm -rf {NESTED_DATA_PATH}

print("\nChecking for unzipped files (showing first 5):")
!ls -l {EXTRACT_DIR} | head -n 6

## 3. (重要) 状態シミュレータによる徹底調査

1つの牌譜ファイルを指定し、イベントごとにシミュレータが認識している全プレイヤーの手牌を詳細に出力します。
このログと実際の牌譜を見比べることで、どのタイミングで手牌の同期がズレるのかを正確に特定できます。

In [ ]:
# 調査したい牌譜ログのファイル名をここに指定してください
# 指定しない場合、dataフォルダからランダムに1つ選ばれます
TARGET_LOG_FILE = ""

import os

if TARGET_LOG_FILE and os.path.exists(f'/content/data/{TARGET_LOG_FILE}'):
    test_file_path = f'/content/data/{TARGET_LOG_FILE}'
    print(f"Investigating specified file: {TARGET_LOG_FILE}")
else:
    print("No specific file targeted, finding a random log file...")
    test_file = !find /content/data -name "*.mjlog" -print -quit
    if not test_file:
        test_file = !find /content/data -name "*.gz" -print -quit

    if test_file:
        test_file_path = test_file[0]
    else:
        test_file_path = None

if test_file_path:
    print(f"Target for investigation: {os.path.basename(test_file_path)}")
    # 調査スクリプトを実行します
    !python -m src.transformer.investigate_state "{test_file_path}"
else:
    print("No .mjlog or .gz files found for investigation in /content/data")

## 4. 全データセットの生成
（原因が特定・修正されるまで、このステップは実行しないでください）

In [ ]:
!python -m src.transformer.generate_data

## 5. AIモデルの学習

In [ ]:
!python -m src.transformer.train_transformer

## 6. 学習済みAIによる予測

In [ ]:
!python -m src.transformer.predict_transformer